In [2]:
import requests

In [3]:
url = 'https://imunizacao-es.saude.gov.br/_search'

In [8]:
username = 'imunizacao_public'
password = 'qlto5t&7r_@+#Tlstigi'

response = requests.get(url, auth=(username, password))
print(response.status_code)
print(response.json())

200
{'took': 7, 'timed_out': False, '_shards': {'total': 30, 'successful': 30, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'desc-imunizacao-v5', '_type': '_doc', '_id': 'b4914f0c-0015-4866-8f35-6111610405d5-i0b0', '_score': 1.0, '_source': {'sistema_origem': 'Novo PNI', 'paciente_endereco_coPais': '10', 'vacina_codigo': '87', 'vacina_fabricante_nome': 'PFIZER', 'vacina_nome': 'COVID-19 PFIZER - COMIRNATY', 'estabelecimento_razaoSocial': 'MUNICIPIO DE CASCAVEL', 'paciente_enumSexoBiologico': 'M', 'paciente_endereco_uf': 'SC', 'data_importacao_rnds': '2022-07-26T14:32:37.000Z', 'paciente_racaCor_valor': 'PARDA', 'vacina_categoria_codigo': '2', 'estabelecimento_valor': '9395156', 'paciente_id': 'ec50147461eb4dec3039c0d0e65340c5938978b512fadf8aaed478ea7be39bb6', 'vacina_lote': 'FN9606', 'data_importacao_datalake': '2022-07-27T18:09:57.000Z', 'estabelecimento_uf': 'PR', '@timestamp': '2022-07-27T21:21:36.507Z', 'pa